In [2]:
import datetime
import time
import os
import sys
import pandas as pd
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_19_10")
from pandas.io import sql
import mysql.connector
import sqlalchemy
import codecs
import numpy as np
import pymysql
import math
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sqlalchemy import types, create_engine

import warnings
warnings.filterwarnings("ignore")
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
from datetime import date, timedelta, datetime
from mysql.connector import errorcode
print('Starting script at: ', datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

Starting script at:  11/10/2021 17:18:52


In [3]:
# clickhouse
from clickhouse_driver import Client
client = Client('192.168.191.15', user = 'ashlenskaya', password = 'uJJG0arPjwg4by4d')


# NewCache
dbv_engine = create_engine("oracle+cx_oracle://itvarw:vr45725h@(DESCRIPTION = (LOAD_BALANCE=on) (FAILOVER=ON) (ADDRESS = (PROTOCOL = TCP)(HOST = 10.42.112.151)(PORT = 1521)) (CONNECT_DATA = (SERVER = DEDICATED) (SERVICE_NAME = dbview)))")

db_config = {'user': 'ashlenskaya', # имя пользователя
'pwd': '4PAMWMjjh86N4Q3w', # пароль
'host': '192.168.191.146',
'port': 9432, # порт подключения
'db': 'itv_api_ng'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'}, echo=False)

In [4]:
def convertList(list):
    separator = "','"
    output = "'"
    for i in range(len(list)):
        item = str(list[i])
        if(i == len(list)-1):
            output+=item+"'"
        else:    
            output+=item+separator
    return output

In [5]:
def find_data(databasename, queryname):
    os.environ["NLS_LANG"] = "RUSSIAN_RUSSIA.AL32UTF8" #Это позволит вывести кириллицу
    MRFS = {
        'core': ('10.50.168.3', '1521', '10.50.168.29', 'sdp.db.core.iptv.rt.ru', '10.50.168.3', '10.50.168.30'),
        'mos': ('10.50.168.3', '1521', '10.50.168.13', 'spine.mos.iptv.rt.ru', '10.50.168.3', '10.50.168.14'),
        'ct': ('10.50.168.3', '1521', '10.50.168.15', 'spine.center.iptv.rt.ru', '10.50.168.3', '10.50.168.16'),
        'vlg': ('10.50.168.3', '1521', '10.50.168.21', 'spine.volga.iptv.rt.ru', '10.50.168.3', '10.50.168.22'),
        'ural': ('10.50.168.3', '1521', '10.50.168.23', 'spine.ural.iptv.rt.ru', '10.50.168.3', '10.50.168.24'),
        'sth': ('10.50.168.3', '1521', '10.50.168.19', 'spine.south.iptv.rt.ru', '10.50.168.3', '10.50.168.20'),
        'sib': ('10.50.168.3', '1521', '10.50.168.25', 'spine.sibir.iptv.rt.ru', '10.50.168.3', '10.50.168.26'),
        'nw': ('10.50.168.3', '1521', '10.50.168.17', 'spine.nw.iptv.rt.ru', '10.50.168.3', '10.50.168.18'),
        'dv': ('10.50.168.3', '1521', '10.50.168.27', 'spine.dv.iptv.rt.ru', '10.50.168.3', '10.50.168.28'),
         }
    oracle_connection = cx_Oracle.connect(('architect_dep/A97z0G9RiNFT2K@(DESCRIPTION_LIST=(load_balance=no)(failover=yes)(DESCRIPTION=(ADDRESS_LIST=(SOURCE_ROUTE=yes)(ADDRESS=(PROTOCOL=tcp)'
                                      +'(HOST={0})(PORT={1}))(ADDRESS=(PROTOCOL=tcp)'
                                      +'(HOST={2})(PORT={1})))(CONNECT_DATA='
                                      +'(SERVICE_NAME={3})))(DESCRIPTION=(ADDRESS_LIST=(SOURCE_ROUTE=yes)(ADDRESS=(PROTOCOL=tcp)'
                                      +'(HOST={4})(PORT={1}))(ADDRESS=(PROTOCOL=tcp)'
                                      +'(HOST={5})(PORT={1})))(CONNECT_DATA='
                                      +'(SERVICE_NAME={3}))))').format(*MRFS[databasename]))
    cur = oracle_connection.cursor()

    
# Варианты запросов  ------------------------------------------------------------------------------------------------------------    


    query1 = '''SELECT  ID as CONTENT_ID, DISCRIMINATOR, NAME, YEAR, IMDB_RATING,IMDB_ID,KINOPOISK_RATING,KINOPOISK_ID
               FROM master.content
               where NAME is not NULL
               AND DISCRIMINATOR IN ('SERIES')
               AND (IS_UNITED_SOURCE IS NULL OR IS_UNITED_SOURCE = 0)'''
    
    query2 = '''SELECT  ID as CONTENT_ID, DISCRIMINATOR, NAME, YEAR, IMDB_RATING,IMDB_ID,KINOPOISK_RATING,KINOPOISK_ID
               FROM master.content
               where NAME is not NULL
               AND DISCRIMINATOR IN ('SERIES', 'VIDEOMOVIE')
               and name not like '%Сезон%'
               and name not like '%Серия%'
               and name not like '%сезон%'
               and name not like '%серия%'
               and name not like '%аудио%'
               and name not like '%Аудио%'
               AND (IS_UNITED_SOURCE IS NULL OR IS_UNITED_SOURCE = 0) '''
    
    
# Варианты запросов  ------------------------------------------------------------------------------------------------------------    

    if queryname == 'query1':
        query = query1 
    elif queryname == 'query2':
        query = query2
    else:
        query = 'select.'
    
 #--------------------------------------------------------------------------------------------------------------------------------
    cur.execute(query)
    result = cur.fetchall()
# -----------------------------------------------------------------------------------------       
    cur.close()
    oracle_connection.close()
    return result

### Выгружаем сериалы SERIES

In [6]:
table1= pd.DataFrame(columns = ['CONTENT_ID', 'DISCRIMINATOR', 'NAME', 'YEAR', 'IMDB_RATING', 'IMDB_ID','KINOPOISK_RATING','KINOPOISK_ID'])
result = []
print('Start at: ', datetime.now().strftime("%d/%m/%Y %H:%M:%S"),'For', 'core', ':')
result = find_data('core', 'query1')
STEP_TABLE = pd.DataFrame(result, columns = ['CONTENT_ID', 'DISCRIMINATOR', 'NAME', 'YEAR', 'IMDB_RATING', 'IMDB_ID','KINOPOISK_RATING','KINOPOISK_ID'])
table1 = pd.concat([table1,STEP_TABLE])
print('Start at: ', datetime.now().strftime("%d/%m/%Y %H:%M:%S"),'core', ': successful')

Start at:  11/10/2021 17:18:55 For core :
Start at:  11/10/2021 17:18:58 core : successful


In [7]:
table1

,CONTENT_ID,DISCRIMINATOR,NAME,YEAR,IMDB_RATING,IMDB_ID,KINOPOISK_RATING,KINOPOISK_ID
0,125637495,SERIES,Про животных и людей,2015,None,None,None,None
1,124972701,SERIES,Незабытая,2020,None,None,0,1402757
2,125666069,SERIES,Шорт-лист,2019,None,None,None,None
3,124094469,SERIES,Шторм поднимается,2020,None,None,None,None
4,124112888,SERIES,Коммандос (2020),2020,7.1,None,0,1296004
...,...,...,...,...,...,...,...,...
3464,123449380,SERIES,Мальчик-Пальчик,2015,None,None,None,None
3465,123857454,SERIES,Звёзды футбола,2017,None,None,None,None
3466,122959308,SERIES,Война с терроризмом,2017,None,None,None,None
3467,123598433,SERIES,Путь до Токио,2020,None,None,None,None


In [8]:
table1.shape

(3469, 8)

In [9]:
table1.query('CONTENT_ID == 75584313')

,CONTENT_ID,DISCRIMINATOR,NAME,YEAR,IMDB_RATING,IMDB_ID,KINOPOISK_RATING,KINOPOISK_ID
1200,75584313,SERIES,Отважные птенцы,2017,6.1,None,0,1171937


In [37]:
def find_mrf_data(databasename, intervalList, queryname):
    os.environ["NLS_LANG"] = "RUSSIAN_RUSSIA.AL32UTF8" #Это позволит вывести кириллицу
    MRFS = {
        'core': ('10.50.168.3', '1521', '10.50.168.29', 'sdp.db.core.iptv.rt.ru', '10.50.168.3', '10.50.168.30'),
        'mos': ('10.50.168.3', '1521', '10.50.168.13', 'spine.mos.iptv.rt.ru', '10.50.168.3', '10.50.168.14'),
        'ct': ('10.50.168.3', '1521', '10.50.168.15', 'spine.center.iptv.rt.ru', '10.50.168.3', '10.50.168.16'),
        'vlg': ('10.50.168.3', '1521', '10.50.168.21', 'spine.volga.iptv.rt.ru', '10.50.168.3', '10.50.168.22'),
        'ural': ('10.50.168.3', '1521', '10.50.168.23', 'spine.ural.iptv.rt.ru', '10.50.168.3', '10.50.168.24'),
        'sth': ('10.50.168.3', '1521', '10.50.168.19', 'spine.south.iptv.rt.ru', '10.50.168.3', '10.50.168.20'),
        'sib': ('10.50.168.3', '1521', '10.50.168.25', 'spine.sibir.iptv.rt.ru', '10.50.168.3', '10.50.168.26'),
        'nw': ('10.50.168.3', '1521', '10.50.168.17', 'spine.nw.iptv.rt.ru', '10.50.168.3', '10.50.168.18'),
        'dv': ('10.50.168.3', '1521', '10.50.168.27', 'spine.dv.iptv.rt.ru', '10.50.168.3', '10.50.168.28'),
         }
    oracle_connection = cx_Oracle.connect(('architect_dep/A97z0G9RiNFT2K@(DESCRIPTION_LIST=(load_balance=no)(failover=yes)(DESCRIPTION=(ADDRESS_LIST=(SOURCE_ROUTE=yes)(ADDRESS=(PROTOCOL=tcp)'
                                      +'(HOST={0})(PORT={1}))(ADDRESS=(PROTOCOL=tcp)'
                                      +'(HOST={2})(PORT={1})))(CONNECT_DATA='
                                      +'(SERVICE_NAME={3})))(DESCRIPTION=(ADDRESS_LIST=(SOURCE_ROUTE=yes)(ADDRESS=(PROTOCOL=tcp)'
                                      +'(HOST={4})(PORT={1}))(ADDRESS=(PROTOCOL=tcp)'
                                      +'(HOST={5})(PORT={1})))(CONNECT_DATA='
                                      +'(SERVICE_NAME={3}))))').format(*MRFS[databasename]))
    cur = oracle_connection.cursor()

    ids = convertList(intervalList)

# Варианты запросов  ------------------------------------------------------------------------------------------------------------    
    query3 = '''SELECT ol.ID,ol.DISCRIMINATOR as PARENT_DISCRIMINATOR,ol.PARENT_ID, c.DISCRIMINATOR as CHILD_DISCRIMINATOR,ol.CHILD_ID, c.name AS CHILD_NAME
                FROM OBJECT_LINK ol
                join master.content c on ol.child_id = c.id
                WHERE ol.parent_id in ({})'''.format(ids)
    
# ------------------------------------------------------------------------------------------------------------    
    query4 = '''SELECT ol.ID,ol.DISCRIMINATOR as PARENT_DISCRIMINATOR,ol.PARENT_ID, c.DISCRIMINATOR as CHILD_DISCRIMINATOR,ol.CHILD_ID, c.name AS CHILD_NAME
                FROM OBJECT_LINK ol
                join master.content c on ol.child_id = c.id
                WHERE  c.DISCRIMINATOR != 'VIDEOMOVIE_DELETED' 
                and ol.parent_id in ({})'''.format(ids)
    
# ------------------------------------------------------------------------------------------------------------    
    query5 = '''SELECT t1.ID AS CONTENT_ID, 
                        t1.NAME AS CONTENT_NAME, 
                        t3.I_ID AS PACKAGE_I_ID, 
                        t3.DISCRIMINATOR AS PACKAGE_DISCRIMINATOR,
                        t3.NAME AS PACKAGE_NAME
                FROM MASTER.CONTENT t1
                JOIN MASTER.OBJECT_LINK t2 ON t2.CHILD_ID = t1.ID
                    AND t2.DISCRIMINATOR IN ( 'SERIESINPACKAGE', 'CONTENTBUNDLEINPACKAGE', 'CONTENTBUNDLEINESTPACKAGE', 'VIDEOMOVIEINESTVODPACKAGE', 'VIDEOMOVIEINVODPACKAGE' )
                    AND ( t2.START_DATE <= SYSDATE OR t2.START_DATE IS NULL )
                    AND ( t2.STOP_DATE >= SYSDATE OR t2.STOP_DATE IS NULL )
                JOIN MASTER.SERVICE_SPEC t3 ON t3.ID = t2.PARENT_ID
                    AND t3.DISCRIMINATOR IN ( 'CONTENTBUNDLEPACKAGE', 'VODPACKAGE', 'ESTVODPACKAGE', 'SERIESPACKAGE', 'ESTCONTENTBUNDLEPACKAGE' )
                WHERE 1 = 1
                    AND t1.DISCRIMINATOR IN ( 'VIDEOMOVIE', 'CONTENTBUNDLE', 'SERIES' )
                    AND t1.IS_PUBLISHED = 1
                    AND ( t1.START_DATE <= SYSDATE OR t1.START_DATE IS NULL )
                    AND ( t1.END_DATE >= SYSDATE OR t1.END_DATE IS NULL )
                    AND t1.ID IN ({})'''.format(ids)
    
# ------------------------------------------------------------------------------------------------------------    
    query6 = '''SELECT t1.I_ID as PACKAGE_I_ID,t1.NAME, t1.DISCRIMINATOR, t6.NAME AS USAGE_RULE, COALESCE( MIN(t5.PRICE), MIN(t4.PRICE), 0 ) AS PRICE
                    FROM MASTER.SERVICE_SPEC t1
                JOIN MASTER.OBJECT_LINK t2 ON t2.CHILD_ID = t1.ID
                    AND t2.DISCRIMINATOR = 'SERVICESPECINPRODUCTOFFER'
                    AND ( t2.START_DATE <= SYSDATE OR t2.START_DATE IS NULL )
                    AND ( t2.STOP_DATE >= SYSDATE OR t2.STOP_DATE IS NULL )
                JOIN MASTER.PRODUCT_OFFER t3 ON t3.ID = t2.PARENT_ID
                    AND ( t3.START_DATE <= SYSDATE OR t3.START_DATE IS NULL )
                    AND ( t3.END_DATE >= SYSDATE OR t3.END_DATE IS NULL )
                LEFT JOIN MASTER.PRICE_LIST t4 ON t4.SSINPO_ID = t2.ID
                    AND ( t4.START_DATE <= SYSDATE OR t4.START_DATE IS NULL )
                    AND ( t4.STOP_DATE >= SYSDATE OR t4.STOP_DATE IS NULL )
                LEFT JOIN MASTER.PRICE_LIST_GRID t5 ON t5.PRICE_LIST_ID = t4.ID
                LEFT JOIN MASTER.CONTENT_USAGE_RULE t6 ON t6.ID = t5.CONTENT_USAGE_RULE_ID
                WHERE 1 = 1
                    AND t1.I_ID IN ({})
                GROUP BY t1.I_ID,
                    t1.NAME,
                    t1.DISCRIMINATOR,
                    t6.NAME'''.format(ids)

# Варианты запросов  ------------------------------------------------------------------------------------------------------------    

    if queryname == 'query3':
        query = query3
    elif queryname == 'query4':
        query = query3
    elif queryname == 'query5':
        query = query5
    elif queryname == 'query6':
        query = query6
    else:
        query = 'select.'
    
 #--------------------------------------------------------------------------------------------------------------------------------
    cur.execute(query)
    result = cur.fetchall()
# -----------------------------------------------------------------------------------------       
    cur.close()
    oracle_connection.close()
    return result

### Выгружаем CONTENTBUNDLE

In [11]:
def partitionResultNames(databaseName, san_ids):
    print(databaseName)
    
    san_ids = list(map(lambda x: str(x), san_ids))
    
    div = divmod(len(san_ids), 1000) # что бы узнать кол-во интервалов, делим длинну массива на 1000, первый элемент содержит кол-во целых интервалов, второй - остаток
    
    round_lengh = (div[0] * 1000) # кол-во элементов массива, кратных 1000 (интервалов)
    
    tail = san_ids[round_lengh+1:] # хвостом будет интервал от кол-ва элементов массива, кратных 1000 + длина последнего с остатком
    head = san_ids[:round_lengh] # началом будет интервал от 0 до кол-ва элементов массива, кратных 1000
    
    a = np.array(head)
    
    multidimensional_list = np.reshape(a, (div[0], 1000)).tolist() # делим на партиции по 1000 
    
    if(len(tail) > 0):
        multidimensional_list.append(tail)
        
    df_for_mrf = pd.DataFrame(columns=['CONTENT_ID', 'PARENT_DISCRIMINATOR', 'PARENT_ID','CHILD_DISCRIMINATOR','CHILD_ID', 'CHILD_NAME'])
    
    # проходимся циклом и записываем результаты в датафрейм
    for partition in multidimensional_list:
        result = find_mrf_data(databaseName, partition, 'query3')
        STEP_TABLE = pd.DataFrame(result, columns = ['CONTENT_ID', 'PARENT_DISCRIMINATOR', 'PARENT_ID','CHILD_DISCRIMINATOR','CHILD_ID', 'CHILD_NAME'])
        df_for_mrf = pd.concat([df_for_mrf, STEP_TABLE])
    print(df_for_mrf.shape, ' df_for_mrf')
    return df_for_mrf

In [12]:
list_df = []
ResultNames= pd.DataFrame(columns=['CONTENT_ID', 'PARENT_DISCRIMINATOR', 'PARENT_ID','CHILD_DISCRIMINATOR','CHILD_ID', 'CHILD_NAME'])
# вызываем ф-цию для санов, сгруппированных по МРФ и записываем в промежуточные таблицы 
san_ids = table1['CONTENT_ID'].unique()
mrf_name = 'core'
# далее конкатенируем все промежуточные таблицы в одну и записываем в таблицу FINISH_RESULT    
ResultNames = pd.concat([ResultNames, partitionResultNames(mrf_name, san_ids)])
print(ResultNames.shape, 'ResultNames')

core
(6056, 6)  df_for_mrf
(6056, 6) ResultNames


In [13]:
ResultNames.head()

,CONTENT_ID,PARENT_DISCRIMINATOR,PARENT_ID,CHILD_DISCRIMINATOR,CHILD_ID,CHILD_NAME
0,15956484,CONTENTBUNDLEINSERIES,12700605,CONTENTBUNDLE,15956466,Южный парк. Сезон 9
1,57723399,CONTENTBUNDLEINSERIES,57722064,CONTENTBUNDLE,57723290,Вавилон-Берлин. Сезон 1
2,25445589,CONTENTBUNDLEINSERIES,25445487,CONTENTBUNDLE,25445379,Назад в СССР
3,19029283,CONTENTBUNDLEINSERIES,12700605,CONTENTBUNDLE,19029066,Южный парк. Сезон 14
4,19033319,CONTENTBUNDLEINSERIES,12700605,CONTENTBUNDLE,19033301,Южный парк. Сезон 15


In [14]:
ResultNames.query('PARENT_ID == 28276294')

,CONTENT_ID,PARENT_DISCRIMINATOR,PARENT_ID,CHILD_DISCRIMINATOR,CHILD_ID,CHILD_NAME
173,69186895,CONTENTBUNDLEINSERIES,28276294,CONTENTBUNDLE,69186879,Власть в ночном городе. Сезон 5
298,28276513,CONTENTBUNDLEINSERIES,28276294,CONTENTBUNDLE,28276203,Власть в ночном городе. Сезон 1
861,41193273,CONTENTBUNDLEINSERIES,28276294,CONTENTBUNDLE,41193259,Власть в ночном городе. Сезон 3
1147,31563725,CONTENTBUNDLEINSERIES,28276294,CONTENTBUNDLE,31563712,Власть в ночном городе. Сезон 2
1167,69187005,CONTENTBUNDLEINSERIES,28276294,CONTENTBUNDLE,69186999,Власть в ночном городе. Сезон 5
1880,50990664,CONTENTBUNDLEINSERIES,28276294,CONTENTBUNDLE,50990529,Власть в ночном городе. Сезон 4
1896,83070383,CONTENTBUNDLEINSERIES,28276294,CONTENTBUNDLE,83070365,Власть в ночном городе. Сезон 6


### Выгружаем VIDEOMOVIE

In [15]:
def partitionResultNames2(databaseName, san_ids):
    print(databaseName)
    
    san_ids = list(map(lambda x: str(x), san_ids))
    
    div = divmod(len(san_ids), 1000) # что бы узнать кол-во интервалов, делим длинну массива на 1000, первый элемент содержит кол-во целых интервалов, второй - остаток
    
    round_lengh = (div[0] * 1000) # кол-во элементов массива, кратных 1000 (интервалов)
    
    tail = san_ids[round_lengh+1:] # хвостом будет интервал от кол-ва элементов массива, кратных 1000 + длина последнего с остатком
    head = san_ids[:round_lengh] # началом будет интервал от 0 до кол-ва элементов массива, кратных 1000
    
    a = np.array(head)
    
    multidimensional_list = np.reshape(a, (div[0], 1000)).tolist() # делим на партиции по 1000 
    
    if(len(tail) > 0):
        multidimensional_list.append(tail)
        
    df_for_mrf = pd.DataFrame(columns=['CONTENT_ID', 'PARENT_DISCRIMINATOR', 'PARENT_ID','CHILD_DISCRIMINATOR','CHILD_ID', 'CHILD_NAME'])
    
    # проходимся циклом и записываем результаты в датафрейм
    for partition in multidimensional_list:
        result = find_mrf_data(databaseName, partition, 'query4')
        STEP_TABLE = pd.DataFrame(result, columns = ['CONTENT_ID', 'PARENT_DISCRIMINATOR', 'PARENT_ID','CHILD_DISCRIMINATOR','CHILD_ID', 'CHILD_NAME'])
        df_for_mrf = pd.concat([df_for_mrf, STEP_TABLE])
    print(df_for_mrf.shape, ' df_for_mrf')
    return df_for_mrf

In [16]:
list_df = []
ResultNames2= pd.DataFrame(columns=['CONTENT_ID', 'PARENT_DISCRIMINATOR', 'PARENT_ID','CHILD_DISCRIMINATOR','CHILD_ID', 'CHILD_NAME'])
# вызываем ф-цию для санов, сгруппированных по МРФ и записываем в промежуточные таблицы 
san_ids = ResultNames['CHILD_ID'].unique()
mrf_name = 'core'
# далее конкатенируем все промежуточные таблицы в одну и записываем в таблицу FINISH_RESULT    
ResultNames2 = pd.concat([ResultNames2, partitionResultNames2(mrf_name, san_ids)])
print(ResultNames2.shape, 'ResultNames2')

core
(125257, 6)  df_for_mrf
(125257, 6) ResultNames2


In [17]:
ResultNames2.sample(5)

,CONTENT_ID,PARENT_DISCRIMINATOR,PARENT_ID,CHILD_DISCRIMINATOR,CHILD_ID,CHILD_NAME
4310,113990577,VIDEOMOVIEINCONTENTBUNDLE,113818763,VIDEOMOVIE,113990572,"Спорим, что такого вы еще не видели? Идея прос..."
43230,125922586,VIDEOMOVIEINCONTENTBUNDLE,125917665,VIDEOMOVIE,125922584,Постучись в мою дверь. Сезон 2. Серия 19
11123,106669649,VIDEOMOVIEINCONTENTBUNDLE,106658804,VIDEOMOVIE,106669531,Звездный крейсер Галактика. Сезон 1. Серия 11
38628,125227447,VIDEOMOVIEINCONTENTBUNDLE,125226134,VIDEOMOVIE,125227446,Царство грибов. Сезон 2. Лечебные грибы
28355,120703087,VIDEOMOVIEINCONTENTBUNDLE,120698568,VIDEOMOVIE,120703086,Он и она. Николай Валуев


In [18]:
ResultNames2.query('PARENT_ID == 31563712')

,CONTENT_ID,PARENT_DISCRIMINATOR,PARENT_ID,CHILD_DISCRIMINATOR,CHILD_ID,CHILD_NAME
717,32091595,VIDEOMOVIEINCONTENTBUNDLE,31563712,VIDEOMOVIE,32091587,Власть в ночном городе. Сезон 2. Серия 4
783,32200238,VIDEOMOVIEINCONTENTBUNDLE,31563712,VIDEOMOVIE,32200230,Власть в ночном городе. Сезон 2. Серия 7
882,32200099,VIDEOMOVIEINCONTENTBUNDLE,31563712,VIDEOMOVIE,32200091,Власть в ночном городе. Сезон 2. Серия 6
893,32422570,VIDEOMOVIEINCONTENTBUNDLE,31563712,VIDEOMOVIE,32422562,Власть в ночном городе. Сезон 2. Серия 9
920,32367413,VIDEOMOVIEINCONTENTBUNDLE,31563712,VIDEOMOVIE,32367405,Власть в ночном городе. Сезон 2. Серия 8
1079,32586540,VIDEOMOVIEINCONTENTBUNDLE,31563712,VIDEOMOVIE,32586532,Власть в ночном городе. Сезон 2. Серия 10
3915,31805401,VIDEOMOVIEINCONTENTBUNDLE,31563712,VIDEOMOVIE,31805393,Власть в ночном городе. Сезон 2. Серия 3
5508,32091791,VIDEOMOVIEINCONTENTBUNDLE,31563712,VIDEOMOVIE,32091783,Власть в ночном городе. Сезон 2. Серия 5
5540,31564700,VIDEOMOVIEINCONTENTBUNDLE,31563712,VIDEOMOVIE,31526487,Власть в ночном городе. Сезон 2. Серия 2
7474,31564699,VIDEOMOVIEINCONTENTBUNDLE,31563712,VIDEOMOVIE,31526369,Власть в ночном городе. Сезон 2. Серия 1


### Выгрузка фильмов и сериалов

In [19]:
table2= pd.DataFrame(columns = ['CONTENT_ID', 'DISCRIMINATOR', 'NAME', 'YEAR', 'IMDB_RATING', 'IMDB_ID','KINOPOISK_RATING','KINOPOISK_ID'])
result = []
print('Start at: ', datetime.now().strftime("%d/%m/%Y %H:%M:%S"),'For', 'core', ':')
result = find_data('core', 'query2')
STEP_TABLE = pd.DataFrame(result, columns = ['CONTENT_ID', 'DISCRIMINATOR', 'NAME', 'YEAR', 'IMDB_RATING', 'IMDB_ID','KINOPOISK_RATING','KINOPOISK_ID'])
table2 = pd.concat([table2,STEP_TABLE])
print('Start at: ', datetime.now().strftime("%d/%m/%Y %H:%M:%S"),'core', ': successful')

Start at:  11/10/2021 17:20:40 For core :
Start at:  11/10/2021 17:21:15 core : successful


In [20]:
table2

,CONTENT_ID,DISCRIMINATOR,NAME,YEAR,IMDB_RATING,IMDB_ID,KINOPOISK_RATING,KINOPOISK_ID
0,125637495,SERIES,Про животных и людей,2015,None,None,None,None
1,124972701,SERIES,Незабытая,2020,None,None,0,1402757
2,125666069,SERIES,Шорт-лист,2019,None,None,None,None
3,124094469,SERIES,Шторм поднимается,2020,None,None,None,None
4,124112888,SERIES,Коммандос (2020),2020,7.1,None,0,1296004
...,...,...,...,...,...,...,...,...
77071,123601398,VIDEOMOVIE,Путь до Токио. Королевы трассы,2020,None,None,None,None
77072,123602242,VIDEOMOVIE,Путь до Токио. Велоспорт,2020,None,None,None,None
77073,123603780,VIDEOMOVIE,"Путь до Токио. Каяк, тройной прыжок и плавание",2020,None,None,None,None
77074,123603943,VIDEOMOVIE,Путь до Токио. Триатлон,2020,None,None,None,None


In [21]:
ex_in = table2[~table2['CONTENT_ID'].isin(ResultNames2['CHILD_ID'])]

In [22]:
ex_in

,CONTENT_ID,DISCRIMINATOR,NAME,YEAR,IMDB_RATING,IMDB_ID,KINOPOISK_RATING,KINOPOISK_ID
0,125637495,SERIES,Про животных и людей,2015,None,None,None,None
1,124972701,SERIES,Незабытая,2020,None,None,0,1402757
2,125666069,SERIES,Шорт-лист,2019,None,None,None,None
3,124094469,SERIES,Шторм поднимается,2020,None,None,None,None
4,124112888,SERIES,Коммандос (2020),2020,7.1,None,0,1296004
...,...,...,...,...,...,...,...,...
77063,122963275,VIDEOMOVIE,Зов души,2012,5.8,None,0,678126
77064,122991656,VIDEOMOVIE,Банда Чикаго,2020,6.1,None,5.612,953620
77068,123782248,VIDEOMOVIE,Уокер Пейн,2006,6,None,0,171736
77069,122685360,VIDEOMOVIE,Стервозные штучки,2009,4.5,None,4.736,409912


In [23]:
ex_in.sample(5)

,CONTENT_ID,DISCRIMINATOR,NAME,YEAR,IMDB_RATING,IMDB_ID,KINOPOISK_RATING,KINOPOISK_ID
38187,125390769,VIDEOMOVIE,Ради семьи. Лидия Чарская,2020,None,None,None,None
48424,110503572,VIDEOMOVIE,Аленький цветочек для чудовища. Дарья Калинина,2020,None,None,None,None
15942,31369617,VIDEOMOVIE,Губка Боб Квадратные Штаны. Проклятье над Бики...,2009,8.1,None,6.959,161268
675,75743098,SERIES,Приключения Незнайки и его друзей,1971,6.3,None,6.825,540639
1379,98979791,SERIES,Поездка за счастьем (Экспресс-командировка),2016,None,None,6.382,986685


In [24]:
ex_in['CONTENT_ID'].nunique()

33135

## Цена

### Вхождение в пакеты

In [25]:
def partition(databaseName, san_ids):
    print(databaseName)
    
    san_ids = list(map(lambda x: str(x), san_ids))
    
    div = divmod(len(san_ids), 1000) # что бы узнать кол-во интервалов, делим длинну массива на 1000, первый элемент содержит кол-во целых интервалов, второй - остаток
    
    round_lengh = (div[0] * 1000) # кол-во элементов массива, кратных 1000 (интервалов)
    
    tail = san_ids[round_lengh+1:] # хвостом будет интервал от кол-ва элементов массива, кратных 1000 + длина последнего с остатком
    head = san_ids[:round_lengh] # началом будет интервал от 0 до кол-ва элементов массива, кратных 1000
    
    a = np.array(head)
    
    multidimensional_list = np.reshape(a, (div[0], 1000)).tolist() # делим на партиции по 1000 
    
    if(len(tail) > 0):
        multidimensional_list.append(tail)
        
    df_for_mrf = pd.DataFrame(columns=['CONTENT_ID', 'CONTENT_NAME', 'PACKAGE_I_ID','PACKAGE_DISCRIMINATOR','PACKAGE_NAME'])
    
    # проходимся циклом и записываем результаты в датафрейм
    for partition in multidimensional_list:
        result = find_mrf_data(databaseName, partition, 'query5')
        STEP_TABLE = pd.DataFrame(result, columns = ['CONTENT_ID', 'CONTENT_NAME', 'PACKAGE_I_ID','PACKAGE_DISCRIMINATOR','PACKAGE_NAME'])
        df_for_mrf = pd.concat([df_for_mrf, STEP_TABLE])
    print(df_for_mrf.shape, ' df_for_mrf')
    return df_for_mrf

In [26]:
list_df = []
Result= pd.DataFrame(columns=['CONTENT_ID', 'CONTENT_NAME', 'PACKAGE_I_ID','PACKAGE_DISCRIMINATOR','PACKAGE_NAME'])
# вызываем ф-цию для санов, сгруппированных по МРФ и записываем в промежуточные таблицы 
san_ids = ex_in['CONTENT_ID'].unique()
mrf_name = 'core'
# далее конкатенируем все промежуточные таблицы в одну и записываем в таблицу FINISH_RESULT    
Result = pd.concat([Result, partition(mrf_name, san_ids)])
print(Result.shape, 'Result')

core
(60099, 5)  df_for_mrf
(60099, 5) Result


In [29]:
Result.sample(10)

,CONTENT_ID,CONTENT_NAME,PACKAGE_I_ID,PACKAGE_DISCRIMINATOR,PACKAGE_NAME
1481,125221125,Два мира,51162749,VODPACKAGE,Категория 2 HD Tizen
2100,55216734,Спарта,121975803,VODPACKAGE,Категория Библиотека Tizen
1515,104885382,UFC Fight Night Abu Dhabi 4,74007260,SERIESPACKAGE,Сериал Для любителей Tizen
1033,98211880,Ночь в музее Эротики,51162749,VODPACKAGE,Категория 2 HD Tizen
1151,105394557,Диета по-итальянски,51174525,VODPACKAGE,Категория 2 Tizen
1343,104308526,Мой шпион,51166947,VODPACKAGE,Категория 4 Tizen
1121,98854454,Human lost: исповедь неполноценного человека,72726400,VODPACKAGE,Категория Для киноманов Tizen
1000,84551819,Вельзевул HD,58448812,VODPACKAGE,Категория Для киноманов
532,95590743,8 класс. Химия,51665002,SERIESPACKAGE,Все сериалы Tizen
698,116653590,FitnessoManiya,51665002,SERIESPACKAGE,Все сериалы Tizen


In [30]:
Result.groupby('PACKAGE_I_ID')['CONTENT_ID'].unique().reset_index()

,PACKAGE_I_ID,CONTENT_ID
0,1000,"[89951737, 89951550, 89951371, 40473454, 37576..."
1,1002,"[81075591, 76544389, 76530317, 73583319, 72710..."
2,1003,"[37576922, 70721911, 70721699, 70721341, 70721..."
3,1004,"[40473454, 13307023, 10773652, 10542847, 10336..."
4,1006,"[75566735, 70732476, 99745982]"
...,...,...
219,122058443,"[124969427, 124079658, 124079483, 124079314, 1..."
220,123905892,"[46853620, 24339594, 72794592, 24121287, 24055..."
221,125221913,"[83440029, 83450931, 83394591, 83388055, 83311..."
222,127785565,"[87191838, 87191294]"


### Цена пакетов

In [31]:
mrflist=['mos','sib','vlg', 'ural','sth','nw','ct','dv']

In [38]:
def partitionPrice(databaseName, san_ids):
    print(databaseName)
    
    san_ids = list(map(lambda x: str(x), san_ids))
    
    div = divmod(len(san_ids), 1000) # что бы узнать кол-во интервалов, делим длинну массива на 1000, первый элемент содержит кол-во целых интервалов, второй - остаток
    
    round_lengh = (div[0] * 1000) # кол-во элементов массива, кратных 1000 (интервалов)
    
    tail = san_ids[round_lengh+1:] # хвостом будет интервал от кол-ва элементов массива, кратных 1000 + длина последнего с остатком
    head = san_ids[:round_lengh] # началом будет интервал от 0 до кол-ва элементов массива, кратных 1000
    
    a = np.array(head)
    
    multidimensional_list = np.reshape(a, (div[0], 1000)).tolist() # делим на партиции по 1000 
    
    if(len(tail) > 0):
        multidimensional_list.append(tail)
        
    df_for_mrf = pd.DataFrame(columns=['PACKAGE_I_ID', 'NAME', 'DISCRIMINATOR','USAGE_RULE', 'PRICE'])
    
    # проходимся циклом и записываем результаты в датафрейм
    for partition in multidimensional_list:
        result = find_mrf_data(databaseName, partition, 'query6')
        STEP_TABLE = pd.DataFrame(result, columns = ['PACKAGE_I_ID','NAME', 'DISCRIMINATOR','USAGE_RULE', 'PRICE'])
        df_for_mrf = pd.concat([df_for_mrf, STEP_TABLE])
    print(df_for_mrf.shape, ' df_for_mrf')
    return df_for_mrf

In [39]:
list_df = []
ResultPrice = pd.DataFrame(columns=['PACKAGE_I_ID','NAME', 'DISCRIMINATOR','USAGE_RULE', 'PRICE'])
# вызываем ф-цию для санов, сгруппированных по МРФ и записываем в промежуточные таблицы 
for i in range (0, len(mrflist)):
    san_ids = Result['PACKAGE_I_ID'].unique()
    mrf_name = mrflist[i]
# далее конкатенируем все промежуточные таблицы в одну и записываем в таблицу FINISH_RESULT    
    ResultPrice = pd.concat([ResultPrice, partitionPrice(mrf_name, san_ids)])
    print(ResultPrice.shape, ' ResultPrice')

mos
(137, 5)  df_for_mrf
(137, 5)  ResultPrice
sib
(137, 5)  df_for_mrf
(274, 5)  ResultPrice
vlg
(137, 5)  df_for_mrf
(411, 5)  ResultPrice
ural
(137, 5)  df_for_mrf
(548, 5)  ResultPrice
sth
(137, 5)  df_for_mrf
(685, 5)  ResultPrice
nw
(137, 5)  df_for_mrf
(822, 5)  ResultPrice
ct
(137, 5)  df_for_mrf
(959, 5)  ResultPrice
dv
(137, 5)  df_for_mrf
(1096, 5)  ResultPrice


In [40]:
ResultPrice.head()

,PACKAGE_I_ID,NAME,DISCRIMINATOR,USAGE_RULE,PRICE
0,82556586,Сериал More.TV Tizen,SERIESPACKAGE,None,0
1,74012133,Сериал Эксклюзивы Tizen,SERIESPACKAGE,None,0
2,51217812,Категория 34 - EST HD - 199 р. Tizen,ESTVODPACKAGE,Коллекция HD OTT,19900
3,51213250,Категория 35 - EST SD - 229 р. Tizen,ESTVODPACKAGE,Коллекция SD OTT,22900
4,51214152,Категория 38 - EST SD - 299 р. Tizen,ESTVODPACKAGE,Коллекция SD OTT,29900


In [41]:
ResultPrice['PACKAGE_I_ID'].nunique()

137

In [42]:
packege_and_price  = Result.merge(ResultPrice, on= 'PACKAGE_I_ID',how = 'left')
packege_and_price['PRICE']  = packege_and_price['PRICE']/100
packege_and_price['PRICE']  =packege_and_price['PRICE'].fillna('not avalible')
packege_and_price=packege_and_price.drop_duplicates(keep='first')
packege_and_price

,CONTENT_ID,CONTENT_NAME,PACKAGE_I_ID,PACKAGE_DISCRIMINATOR,PACKAGE_NAME,NAME,DISCRIMINATOR,USAGE_RULE,PRICE
0,92711353,Сваты,5000,SERIESPACKAGE,Все сериалы,NaN,NaN,NaN,not avalible
1,99815864,Братья в довесок (по версии Кураж-Бамбей),5000,SERIESPACKAGE,Все сериалы,NaN,NaN,NaN,not avalible
2,99813829,Зона дискомфорта,5000,SERIESPACKAGE,Все сериалы,NaN,NaN,NaN,not avalible
3,99812076,Я могу уничтожить тебя,5000,SERIESPACKAGE,Все сериалы,NaN,NaN,NaN,not avalible
4,99810563,Кровавый след,5000,SERIESPACKAGE,Все сериалы,NaN,NaN,NaN,not avalible
...,...,...,...,...,...,...,...,...,...
421784,124159765,Охотник на демонов. Дмитрий Шелег,103407231,ESTVODPACKAGE,249 Аудио Покупка,249 Аудио Покупка,ESTVODPACKAGE,Коллекция SD OTT,249
421792,124159775,Весна умирает осенью. Вера Арье,103407231,ESTVODPACKAGE,249 Аудио Покупка,249 Аудио Покупка,ESTVODPACKAGE,Коллекция SD OTT,249
421800,124161061,"Говори так! Как выступать уверенно, не тупить ...",103407262,ESTVODPACKAGE,299 Аудио Покупка,299 Аудио Покупка,ESTVODPACKAGE,Коллекция SD OTT,299
421808,124162148,Ковен озера Шамплейн. Анастасия Гор,103407231,ESTVODPACKAGE,249 Аудио Покупка,249 Аудио Покупка,ESTVODPACKAGE,Коллекция SD OTT,249


In [43]:
packege_and_min_price = packege_and_price.groupby(['CONTENT_ID','DISCRIMINATOR'])['PRICE'].min().reset_index()
packege_and_min_price

,CONTENT_ID,DISCRIMINATOR,PRICE
0,1743908,ESTVODPACKAGE,199.0
1,1743908,VODPACKAGE,0.0
2,1891957,ESTVODPACKAGE,249.0
3,1891957,VODPACKAGE,0.0
4,1976278,ESTVODPACKAGE,249.0
...,...,...,...
22226,127218135,SERIESPACKAGE,0.0
22227,127220166,VODPACKAGE,0.0
22228,127220573,VODPACKAGE,0.0
22229,127233985,VODPACKAGE,0.0


### Соединяем таблицы

#### Таблица с фильмами

In [44]:
ex_in.head(3)

,CONTENT_ID,DISCRIMINATOR,NAME,YEAR,IMDB_RATING,IMDB_ID,KINOPOISK_RATING,KINOPOISK_ID
0,125637495,SERIES,Про животных и людей,2015,None,None,None,None
1,124972701,SERIES,Незабытая,2020,None,None,0,1402757
2,125666069,SERIES,Шорт-лист,2019,None,None,None,None


In [45]:
FR = ex_in[['CONTENT_ID','NAME','YEAR','IMDB_ID','KINOPOISK_ID']].merge(packege_and_min_price, on ='CONTENT_ID')

In [46]:
FR

,CONTENT_ID,NAME,YEAR,IMDB_ID,KINOPOISK_ID,DISCRIMINATOR,PRICE
0,125637495,Про животных и людей,2015,None,None,SERIESPACKAGE,0.0
1,124972701,Незабытая,2020,None,1402757,SERIESPACKAGE,0.0
2,125666069,Шорт-лист,2019,None,None,SERIESPACKAGE,0.0
3,124094469,Шторм поднимается,2020,None,None,SERIESPACKAGE,0.0
4,124112888,Коммандос (2020),2020,None,1296004,SERIESPACKAGE,0.0
...,...,...,...,...,...,...,...
22226,123782248,Уокер Пейн,2006,None,171736,ESTVODPACKAGE,299.0
22227,123782248,Уокер Пейн,2006,None,171736,VODPACKAGE,0.0
22228,122685360,Стервозные штучки,2009,None,409912,ESTVODPACKAGE,299.0
22229,122685360,Стервозные штучки,2009,None,409912,VODPACKAGE,0.0


In [47]:
FR['KINOPOISK_LINK'] ="https://www.kinopoisk.ru/film/" + FR['KINOPOISK_ID'].map(str)

FR['WINK_LINK'] = "https://wink.rt.ru/media_items/" + FR['CONTENT_ID'].map(str)

FR['IMDB_LINK'] ="https://www.imdb.com/title/" + FR['IMDB_ID'].map(str)

FR = FR.replace('https://www.imdb.com/title/None', '-')

FR = FR.replace('https://www.kinopoisk.ru/film/None', '-')

FR = FR.replace('https://www.kinopoisk.ru/film/None', '-')

FR = FR.replace('https://wink.rt.ru/media_items/None', '-')

In [48]:
FR.query('CONTENT_ID==87740195')

,CONTENT_ID,NAME,YEAR,IMDB_ID,KINOPOISK_ID,DISCRIMINATOR,PRICE,KINOPOISK_LINK,WINK_LINK,IMDB_LINK
525,87740195,Пиноккио (2014),2014,None,843778,SERIESPACKAGE,0.0,https://www.kinopoisk.ru/film/843778,https://wink.rt.ru/media_items/87740195,-


In [49]:
FR.query('CONTENT_ID==87723995')

,CONTENT_ID,NAME,YEAR,IMDB_ID,KINOPOISK_ID,DISCRIMINATOR,PRICE,KINOPOISK_LINK,WINK_LINK,IMDB_LINK


In [50]:
FR.query('NAME == "Пиноккио (2014)" ')

,CONTENT_ID,NAME,YEAR,IMDB_ID,KINOPOISK_ID,DISCRIMINATOR,PRICE,KINOPOISK_LINK,WINK_LINK,IMDB_LINK
525,87740195,Пиноккио (2014),2014,None,843778,SERIESPACKAGE,0.0,https://www.kinopoisk.ru/film/843778,https://wink.rt.ru/media_items/87740195,-


In [51]:
FR

,CONTENT_ID,NAME,YEAR,IMDB_ID,KINOPOISK_ID,DISCRIMINATOR,PRICE,KINOPOISK_LINK,WINK_LINK,IMDB_LINK
0,125637495,Про животных и людей,2015,None,None,SERIESPACKAGE,0.0,-,https://wink.rt.ru/media_items/125637495,-
1,124972701,Незабытая,2020,None,1402757,SERIESPACKAGE,0.0,https://www.kinopoisk.ru/film/1402757,https://wink.rt.ru/media_items/124972701,-
2,125666069,Шорт-лист,2019,None,None,SERIESPACKAGE,0.0,-,https://wink.rt.ru/media_items/125666069,-
3,124094469,Шторм поднимается,2020,None,None,SERIESPACKAGE,0.0,-,https://wink.rt.ru/media_items/124094469,-
4,124112888,Коммандос (2020),2020,None,1296004,SERIESPACKAGE,0.0,https://www.kinopoisk.ru/film/1296004,https://wink.rt.ru/media_items/124112888,-
...,...,...,...,...,...,...,...,...,...,...
22226,123782248,Уокер Пейн,2006,None,171736,ESTVODPACKAGE,299.0,https://www.kinopoisk.ru/film/171736,https://wink.rt.ru/media_items/123782248,-
22227,123782248,Уокер Пейн,2006,None,171736,VODPACKAGE,0.0,https://www.kinopoisk.ru/film/171736,https://wink.rt.ru/media_items/123782248,-
22228,122685360,Стервозные штучки,2009,None,409912,ESTVODPACKAGE,299.0,https://www.kinopoisk.ru/film/409912,https://wink.rt.ru/media_items/122685360,-
22229,122685360,Стервозные штучки,2009,None,409912,VODPACKAGE,0.0,https://www.kinopoisk.ru/film/409912,https://wink.rt.ru/media_items/122685360,-


In [52]:
FR=FR.drop_duplicates(keep='first')

In [71]:
FR.sample(10)

,CONTENT_ID,NAME,YEAR,IMDB_ID,KINOPOISK_ID,DISCRIMINATOR,PRICE,KINOPOISK_LINK,WINK_LINK,IMDB_LINK,Цена за покупку,Цена за аренду
18750,113428464,Батя,2020,None,1326397,ESTVODPACKAGE,349.0,https://www.kinopoisk.ru/film/1326397,https://wink.rt.ru/media_items/113428464,-,349,
14805,126524452,Террор (1963),1963,None,4229,VODPACKAGE,0.0,https://www.kinopoisk.ru/film/4229,https://wink.rt.ru/media_items/126524452,-,,0
2322,125776947,Лицо войны. Алексей Бобл,2020,None,None,ESTVODPACKAGE,129.0,-,https://wink.rt.ru/media_items/125776947,-,129,
10322,86692241,"Ты, только ты",2016,None,1002153,VODPACKAGE,0.0,https://www.kinopoisk.ru/film/1002153,https://wink.rt.ru/media_items/86692241,-,,0
3292,24797131,Последний король Шотландии,2006,None,156002,ESTVODPACKAGE,149.0,https://www.kinopoisk.ru/film/156002,https://wink.rt.ru/media_items/24797131,-,149,
18874,113426655,Лавка бесконечности (сборник). Роберт Шекли,2020,None,None,ESTVODPACKAGE,249.0,-,https://wink.rt.ru/media_items/113426655,-,249,
7690,92669980,Полоски зебры,2013,None,669903,ESTVODPACKAGE,299.0,https://www.kinopoisk.ru/film/669903,https://wink.rt.ru/media_items/92669980,-,299,
20279,118148543,Тёмные пути. Андрей Васильев,2020,None,None,ESTVODPACKAGE,249.0,-,https://wink.rt.ru/media_items/118148543,-,249,
1757,117876612,ПЛАНЕТКА,2021,None,None,SERIESPACKAGE,0.0,-,https://wink.rt.ru/media_items/117876612,-,,
9084,78441341,Майор,2013,None,675498,VODPACKAGE,0.0,https://www.kinopoisk.ru/film/675498,https://wink.rt.ru/media_items/78441341,-,,0


In [55]:
FR.to_excel('Выгрузка для LiveLib2.xlsx', index = False)